In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [2]:
base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape

(4057201, 16)

In [3]:
base_grande.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0
1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0
2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0
3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0
4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0


In [4]:
ind = []
for i in range (0,4057201) : 
    ind.append(i)
base_grande['ind'] = ind


In [5]:
base_grande.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri,ind
0,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0,0
1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0,1
2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0,2
3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0,3
4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0,4


In [6]:
liste_diff_patho3 = base_grande['patho_niv3'].unique()
liste_diff_patho2 = base_grande['patho_niv2'].unique()
liste_diff_patho1 = base_grande['patho_niv1'].unique()

In [7]:
#creation de la colonne cancer pour simplifier la base
"""
set_cancer = set()
for i in patho_3_diff : 
    i.split()
    if 'Cancer' in i or 'cancer' in i :
        set_cancer.add(i)
    else :
        continue
"""
base_grande = base_grande.dropna()

In [8]:
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    base_grande.loc[:,f"{nom_de_maladie}_ind"] = temporaire

for i in liste_diff_patho1 : 
    creation_indicatrice(colonne_de_depart = base_grande['patho_niv1'], nom_de_maladie = i)
liste_var = base_grande.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)
indicatrice_patho_niv3 = liste_bis
liste_auxil = base_grande['cla_age_5'].unique()
inter_excl = ('95et+', 'tsage', np.nan)

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)
classe_age = []
for i in base_grande['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)
base_grande.loc[:,'classe_age'] = classe_age
base_grande.head()

,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,...,Autres affections de longue durée (dont 31 et 32)_ind,Cancers_ind,Maladies cardio-neurovasculaires_ind,Maladies du foie ou du pancréas (hors mucoviscidose)_ind,Diabète_ind,"Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)_ind",Maladies respiratoires chroniques (hors mucoviscidose)_ind,Maternité (avec ou sans pathologies)_ind,Insuffisance rénale chronique terminale_ind,classe_age
5588,2017.0,Traitements du risque vasculaire (hors patholo...,Traitements antihypertenseurs (hors pathologies),Traitements antihypertenseurs (hors pathologies),top_FAntiHTA_med,00-04,1.0,27,999,10.0,...,0,0,0,0,0,0,0,0,0,2.0
5592,2017.0,Traitements du risque vasculaire (hors patholo...,Traitements antihypertenseurs (hors pathologies),Traitements antihypertenseurs (hors pathologies),top_FAntiHTA_med,00-04,1.0,32,62,20.0,...,0,0,0,0,0,0,0,0,0,2.0
5593,2017.0,Traitements du risque vasculaire (hors patholo...,Traitements antihypertenseurs (hors pathologies),Traitements antihypertenseurs (hors pathologies),top_FAntiHTA_med,00-04,1.0,32,999,50.0,...,0,0,0,0,0,0,0,0,0,2.0
5594,2017.0,Traitements du risque vasculaire (hors patholo...,Traitements antihypertenseurs (hors pathologies),Traitements antihypertenseurs (hors pathologies),top_FAntiHTA_med,00-04,1.0,44,57,10.0,...,0,0,0,0,0,0,0,0,0,2.0
5595,2017.0,Traitements du risque vasculaire (hors patholo...,Traitements antihypertenseurs (hors pathologies),Traitements antihypertenseurs (hors pathologies),top_FAntiHTA_med,00-04,1.0,52,44,10.0,...,0,0,0,0,0,0,0,0,0,2.0


In [9]:
#on refait ce qu'on avait sur le main pour pouvoir avoir les memes variables,
#la meme taille initiale, les memes individus pour pouvoir voir ou ca coince

In [10]:
#il s'avere que prendre un set pour faire des index ne marche pas donc on prend une liste
"""
liste_cancer = list(set_cancer)
liste_cancer_ind = []
for i in liste_cancer : 
    i = i+"_ind"
    liste_cancer_ind.append(i)
"""

'\nliste_cancer = list(set_cancer)\nliste_cancer_ind = []\nfor i in liste_cancer : \n    i = i+"_ind"\n    liste_cancer_ind.append(i)\n'

In [11]:
#base_grande['Cancer'] = base_grande[liste_cancer_ind].sum(axis = 1)

In [12]:
base_grande = base_grande.drop(['top', 'cla_age_5', 'patho_niv2', 'patho_niv3', 'annee', 
                                'libelle_classe_age', 'sexe', 'libelle_sexe'], axis = 1)

In [13]:
liste_var = base_grande.columns

In [14]:
print(liste_var)

Index(['patho_niv1', 'region', 'dept', 'ntop', 'npop', 'prev',
       'niveau_prioritaire', 'tri', 'ind',
       'Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire_ind',
       'Séjours en hospitalisation complète pour prise en charge de la Covid-19_ind',
       'Traitements du risque vasculaire (hors pathologies)_ind',
       'Maladies inflammatoires ou rares ou VIH ou SIDA_ind',
       'Maladies neurologiques ou dégénératives_ind',
       'Traitements psychotropes (hors pathologies)_ind',
       'Traitement antalgique ou anti-inflammatoire (hors pathologies, traitements, maternité ou hospitalisations)_ind',
       'Maladies psychiatriques_ind', 'nan_ind',
       'Autres affections de longue durée (dont 31 et 32)_ind', 'Cancers_ind',
       'Maladies cardio-neurovasculaires_ind',
       'Maladies du foie ou du pancréas (hors mucoviscidose)_ind',
       'Diabète_ind',
       'Hospitalisations ponctuelles (avec ou sans p

In [15]:
df_model_cancer = base_grande[['ind', 'dept', 'classe_age', 'Cancers_ind']]

In [16]:
df_model_cancer.head()

,ind,dept,classe_age,Cancers_ind
5588,5588,999,2.0,0
5592,5592,62,2.0,0
5593,5593,999,2.0,0
5594,5594,57,2.0,0
5595,5595,44,2.0,0


In [17]:
base_grande = base_grande[base_grande['patho

SyntaxError: unterminated string literal (detected at line 1) (2068925053.py, line 1)